In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! tar -xvf '/Downloads/hi_fi_tts_v0.tar.gz' -C '/datasets/hi_fi_tts_v0/'
! cd '/datasets/hi_fi_tts_v0/'

In [ ]:
! pip install audiofile
! pip install pydub

In [ ]:
from pathlib import Path
import os
from tqdm import tqdm
import audiofile
from pydub import AudioSegment
import json

In [ ]:
# coverts flac files to wavs and removes flac

def convert_to_wavs(audio_path):
  flacs = [f for f in audio_path.iterdir() if f.suffix == '.flac']
  for flac in tqdm(flacs):
    wav_name = str(flac.absolute()).split('.')[0] + '.wav'
    flac_audio = AudioSegment.from_file(flac, "flac")
    flac_audio.export(wav_name, format="wav")
    os.remove(flac)

In [ ]:
# remove other audio and files

def remove_other(dataset_path):
  for path in dataset_path.iterdir():
    if path.suffix == '.json' and 'other' in path.name:
      os.remove(path)

  audio_path = dataset_path / 'audio'
  for path in audio_path.iterdir():
    if 'other' in path.name:
      os.remove(path)

In [54]:
# creates meta.csv and fills it with data of the dataset
def create_meta_txt(dataset_path, csv_path):
  jsons = [f for f in dataset_path.iterdir() if f.suffix == '.json']

  with open(csv_path, 'rw') as f:
    for json_file in jsons:
      data = read_json(json_file)
      for wav, text, speaker in data:
        f.write(f'{str(wav)}|{text}|{speaker}\n')


# reads json row by row and collects the data
def read_json(json_path):
  data = []
  speaker = get_speaker(json_path)

  with open(json_path, 'r', newline="\n") as file:
    for row in file:
      data = json.load(row)
      file_path = data['audio_filepath'].split('.')[0] + '.wav'
      text = data['text_normalized']
      if Path(file_path).exists():
        data.append((file_path, text, speaker))
      else:
        print(f'File not found for {file_path}')
  return data


# returns speaker id
def get_speaker(json_file):
  for speaker in SPEAKERS:
    if speaker in json_file.name:
      return speaker
    else:
      print(f'Speaker not found for {json_file.name}')
      return None


In [ ]:
readers = ['9017', '6097', '92']

# SPEAKER_ID = dict(zip(readers, [1, 2, 3]))
SPEAKERS = readers

dataset_path = Path('~/datasets/hi_fi_tts_v0').expanduser()

audio_path = dataset_path / 'audio'
transcript_path = dataset_path
csv_path = dataset_path / 'meta.txt'
# wavs_path = dataset_path / 'wavs'
# wavs_path.mkdir(exist_ok=True)

remove_other(dataset_path)
convert_to_wavs(audio_path)
create_meta_txt(csv_path)
